Kelton McNair - 992018683
CS 601 - Machine Learning
Spring Semester Project 2025
Dr. Afshar
Random Forest Classifier
Dataset: Adult Or letter-recognition?

In [8]:
# pip install ucimlrepo

# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# adult = fetch_ucirepo(id=2) 
  
# # data (as pandas dataframes) 
# X = adult.data.features 
# y = adult.data.targets 
  
# # metadata 
# print(adult.metadata) 
  
# # variable information 
# print(adult.variables) 



In [9]:
# from ucimlrepo import fetch_ucirepo 
  
# # fetch dataset 
# letter_recognition = fetch_ucirepo(id=59) 
  
# # data (as pandas dataframes) 
# X = letter_recognition.data.features 
# y = letter_recognition.data.targets 
  
# # metadata 
# print(letter_recognition.metadata) 
  
# # variable information 
# print(letter_recognition.variables) 


In [10]:
import numpy as np
import pandas as pd

#not sure what all I need for comparative evaluation sklearn model yet
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from numpy.random import default_rng

In [11]:
data = pd.read_csv("adult/adult.data")


In [20]:
data.shape

(32560, 15)

In [21]:
data.head()
# Need to add column names


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [16]:
test = pd.read_csv("adult/adult.test")


In [22]:
test.shape

(16281, 1)

In [17]:
test.head()

,,,,,,,,,,,,,,|1x3 Cross validator
25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [26]:
# does not work, not a csv names = pd.read_csv("adult/adult.names")

In [25]:
with open ("adult/adult.names", "r") as file:
    test = file.read()
print(test)

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

In [ ]:
"age", "workclass", "fnlwgt", "education", "education-num", "marital-status", "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"???????

In [28]:
with open ("adult/index", "r") as file:
    test = file.read()
print(test)

Index of adult

02 Dec 1996      140 Index
10 Aug 1996  3974305 adult.data
10 Aug 1996     4267 adult.names
10 Aug 1996  2003153 adult.test



In [29]:
with open ("adult/old.adult.names", "r") as file:
    test = file.read()
print(test)

1. Title of Database: adult
2. Sources:
   (a) Original owners of database (name/phone/snail address/email address)
       US Census Bureau.
   (b) Donor of database (name/phone/snail address/email address)
       Ronny Kohavi and Barry Becker, 
       Data Mining and Visualization
       Silicon Graphics.
       e-mail: ronnyk@sgi.com
   (c) Date received (databases may change over time without name change!)
       05/19/96
3. Past Usage:
   (a) Complete reference of article where it was described/used
        @inproceedings{kohavi-nbtree,
           author={Ron Kohavi},
           title={Scaling Up the Accuracy of Naive-Bayes Classifiers: a 
                  Decision-Tree Hybrid},
           booktitle={Proceedings of the Second International Conference on
                      Knowledge Discovery and Data Mining},
           year = 1996,
           pages={to appear}}
   (b) Indication of what attribute(s) were being predicted 
       Salary greater or less than 50,000.
   (b) Indica